In [2]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [5]:
leagues = {1: ['4','Spanish-ACB'],
           2: ['6','Italian-Lega-Basket-Serie-A'],
           3: ['7', 'Turkish-BSL'],
           4: ['8', 'Greek-HEBA-A1'],
           5: ['12', 'French-Jeep-Elite'],
           6: ['18', 'Adriatic-League-Liga-ABA'],
           7: ['35','VTB-United-League']
          }

In [6]:
roles=['PG','SG','SF','PF','C']

In [7]:
def get_basic(n):
    print(leagues[n][1])
    data=pd.DataFrame()
    for i in range(2004,2023):
        test=0
        while test==0:
            try:
                url = 'https://basketball.realgm.com/international/league/'+leagues[n][0]+'/'+leagues[n][1]+'/team-stats/'+str(i)+'/Totals/Team_Totals'
                html = urlopen(url)
                soup = BeautifulSoup(html, features="lxml")
                test=1
            except:
                continue
            break
        try:
            table =soup.select('table')[0]
            rows = table.find_all('tr')
            frame= []
            output = []
            for row in rows:
                cols = row.find_all('th')
                cols = [item.text.strip() for item in cols]
                frame.append([item for item in cols if item])
            columns=frame[0][1:]
            columns.insert(0, '#')
            for row in rows:
                cols = row.find_all('td')
                cols = [item.text.strip() for item in cols]
                output.append([item for item in cols if item])
            output=pd.DataFrame.from_records(output,columns=columns)
            output.drop(columns='#',inplace=True)
            output=output[output.index!=0]
            output['Year']=str(str(i-1)+'-'+str(i))
            data=data.append(output)
        except:
            continue
    data['Code']=data['Team'].apply(lambda x: x[:3].upper())
    data['id']=data['Team']+data['Year']    
    return data

In [8]:
def get_advanced(n):
    print(leagues[n][1])
    data=pd.DataFrame()
    for i in range(2004,2023):
        test=0
        while test==0:
            try:
                url = 'https://basketball.realgm.com/international/league/'+leagues[n][0]+'/'+leagues[n][1]+'/team-stats/'+str(i)+'/Advanced_Stats/Team_Totals'
                html = urlopen(url)
                soup = BeautifulSoup(html, features="lxml")
                test=1
            except:
                continue
            break
        try:
            table =soup.select('table')[0]
            rows = table.find_all('tr')
            frame= []
            output = []
            for row in rows:
                cols = row.find_all('th')
                cols = [item.text.strip() for item in cols]
                frame.append([item for item in cols if item])
            columns=frame[0][1:]
            columns.insert(0, '#')
            for row in rows:
                cols = row.find_all('td')
                cols = [item.text.strip() for item in cols]
                output.append([item for item in cols if item])
            output=pd.DataFrame.from_records(output,columns=columns)
            output.drop(columns='#',inplace=True)
            output=output[output.index!=0]
            output['Year']=str(str(i-1)+'-'+str(i))
            data=data.append(output)
        except:
            continue
    data['Code']=data['Team'].apply(lambda x: x[:3].upper())
    data['id']=data['Team']+data['Year']    
    return data

In [9]:
def get_players(n):
    print(leagues[n][1])
    data=pd.DataFrame()
    for i in range(2004,2023):
        for k in roles:
            for j in range(1,5):
                test=0
                while test==0:
                    try:
                        url = 'https://basketball.realgm.com/international/league/'+leagues[n][0]+'/'+leagues[n][1]+'/stats/'+str(i)+'/Totals/All/All/points/'+k+'/desc'+str(j)
                        html = urlopen(url)
                        soup = BeautifulSoup(html, features="lxml")
                        test=1
                    except:
                        continue
                    break
                try:
                    table =soup.select('table')[0]
                    rows = table.find_all('tr')
                    frame= []
                    output = []
                    for row in rows:
                        cols = row.find_all('th')
                        cols = [item.text.strip() for item in cols]
                        frame.append([item for item in cols if item])
                    columns=frame[0][1:]
                    columns.insert(0, '#')
                    for row in rows:
                        cols = row.find_all('td')
                        cols = [item.text.strip() for item in cols]
                        output.append([item for item in cols if item])
                    output=pd.DataFrame.from_records(output,columns=columns)
                    output.drop(columns='#',inplace=True)
                    output=output[output.index!=0]
                    output['Year']=str(str(i-1)+'-'+str(i))
                    output['Role']=k
                    data=data.append(output)
                except:
                    continue
    return data

In [14]:
def get_players_adv(n):
    print(leagues[n][1])
    data=pd.DataFrame()
    for i in range(2004,2023):
        for j in range(1,5):
            test=0
            while test==0:
                try:
                    url = 'https://basketball.realgm.com/international/league/'+leagues[n][0]+'/'+leagues[n][1]+'/stats/'+str(i)+'/Advanced_Stats/All/All/per/All/asc/'+str(j)
                    html = urlopen(url)
                    soup = BeautifulSoup(html, features="lxml")
                    test=1
                except:
                    continue
                break
            try:
                table =soup.select('table')[0]
                rows = table.find_all('tr')
                frame= []
                output = []
                for row in rows:
                    cols = row.find_all('th')
                    cols = [item.text.strip() for item in cols]
                    frame.append([item for item in cols if item])
                columns=frame[0][1:]
                columns.insert(0, '#')
                for row in rows:
                    cols = row.find_all('td')
                    cols = [item.text.strip() for item in cols]
                    output.append([item for item in cols if item])
                output=pd.DataFrame.from_records(output,columns=columns)
                output.drop(columns='#',inplace=True)
                output=output[output.index!=0]
                output['Year']=str(str(i-1)+'-'+str(i))
                data=data.append(output)
            except:
                continue
    return data

In [127]:
def join_teams(basic,advanced):
    teams=basic.merge(advanced,right_on='id',left_on='id')
    teams.drop(columns=['Year_y','Code_y','Team_y'],inplace=True)
    teams.rename(columns={'Year_x':'Year','Code_x':'Code','Team_x':'Team'},inplace=True)
    cols = teams.columns.drop(['Team','Year','Code','id'])
    teams[cols]=teams[cols].apply(lambda x: x.str.replace(',',''))
    teams[cols] = teams[cols].apply(pd.to_numeric, errors='coerce')
    return teams

In [128]:
basic_esp=get_basic(1)
advanced_esp=get_advanced(1)
team_esp=join_teams(basic_esp,advanced_esp)
team_esp.to_csv('team_esp.csv',index=False)

Spanish-ACB


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The f

Spanish-ACB


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The f

NameError: name 'join_team' is not defined

In [130]:
basic_it=get_basic(2)
advanced_it=get_advanced(2)
team_it=join_teams(basic_it,advanced_it)
team_it.to_csv('team_it.csv',index=False)

Italian-Lega-Basket-Serie-A


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The f

Italian-Lega-Basket-Serie-A


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The f

In [131]:
basic_tur=get_basic(3)
advanced_tur=get_advanced(3)
team_tur=join_teams(basic_tur,advanced_tur)
team_tur.to_csv('team_tur.csv',index=False)

Turkish-BSL


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The f

Turkish-BSL


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The f

In [132]:
basic_gr=get_basic(4)
advanced_gr=get_advanced(4)
team_gr=join_teams(basic_gr,advanced_gr)
team_gr.to_csv('team_gr.csv',index=False)

Greek-HEBA-A1


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The f

Greek-HEBA-A1


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The f

In [133]:
basic_fr=get_basic(5)
advanced_fr=get_advanced(5)
team_fr=join_teams(basic_fr,advanced_fr)
team_fr.to_csv('team_fr.csv',index=False)

French-Jeep-Elite


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The f

French-Jeep-Elite


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The f

In [134]:
basic_adr=get_basic(6)
advanced_adr=get_advanced(6)
team_adr=join_teams(basic_adr,advanced_adr)
team_adr.to_csv('team_adr.csv',index=False)

Adriatic-League-Liga-ABA


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The f

Adriatic-League-Liga-ABA


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The f

In [135]:
basic_rus=get_basic(7)
advanced_rus=get_advanced(7)
team_rus=join_teams(basic_rus,advanced_rus)
team_rus.to_csv('team_rus.csv',index=False)

VTB-United-League


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\3643650661.py:34: FutureWarning: The f

VTB-United-League


C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_29248\2954965646.py:34: FutureWarning: The f

In [146]:
pd.pivot_table(team_rus,values=['AST%','STL%','BLK%','ORtg','DRtg'], index=['Year'],aggfunc='mean')

,AST%,BLK%,DRtg,ORtg,STL%
Year,,,,,
2011-2012,50.336364,6.109091,107.240909,104.627273,9.500000
2012-2013,54.230000,6.275000,105.435000,104.200000,10.020000
2013-2014,61.700000,5.475000,109.795000,108.995000,9.760000
2014-2015,63.450000,6.500000,109.493750,108.543750,9.543750
2015-2016,63.168750,6.412500,110.506250,109.150000,9.175000
2016-2017,65.230769,5.923077,110.930769,109.592308,9.246154
2017-2018,63.953846,6.123077,113.369231,112.761538,9.230769
2018-2019,64.342857,6.635714,112.978571,111.907143,9.457143
2019-2020,65.669231,5.969231,114.400000,114.369231,8.469231


In [45]:
players_esp=get_players(1)
cols = players_esp.columns.drop(['Team','Year','Player','Role'])
players_esp[cols]=players_esp[cols].apply(lambda x: x.str.replace(',',''))
players_esp[cols] = players_esp[cols].apply(pd.to_numeric, errors='coerce')

Spanish-ACB


C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

In [46]:
players_it=get_players(2)
cols = players_it.columns.drop(['Team','Year','Player','Role'])
players_it[cols]=players_it[cols].apply(lambda x: x.str.replace(',',''))
players_it[cols] = players_it[cols].apply(pd.to_numeric, errors='coerce')

Italian-Lega-Basket-Serie-A


C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

In [47]:
players_tur=get_players(3)
cols = players_tur.columns.drop(['Team','Year','Player','Role'])
players_tur[cols]=players_tur[cols].apply(lambda x: x.str.replace(',',''))
players_tur[cols] = players_tur[cols].apply(pd.to_numeric, errors='coerce')

Turkish-BSL


C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

In [48]:
players_gr=get_players(4)
cols = players_gr.columns.drop(['Team','Year','Player','Role'])
players_gr[cols]=players_gr[cols].apply(lambda x: x.str.replace(',',''))
players_gr[cols] = players_gr[cols].apply(pd.to_numeric, errors='coerce')

Greek-HEBA-A1


C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

In [49]:
players_fr=get_players(5)
cols = players_fr.columns.drop(['Team','Year','Player','Role'])
players_fr[cols]=players_fr[cols].apply(lambda x: x.str.replace(',',''))
players_fr[cols] = players_fr[cols].apply(pd.to_numeric, errors='coerce')

French-Jeep-Elite


C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

In [50]:
players_adr=get_players(6)
cols = players_adr.columns.drop(['Team','Year','Player','Role'])
players_adr[cols]=players_adr[cols].apply(lambda x: x.str.replace(',',''))
players_adr[cols] = players_adr[cols].apply(pd.to_numeric, errors='coerce')

Adriatic-League-Liga-ABA


C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

In [51]:
players_rus=get_players(7)
cols = players_rus.columns.drop(['Team','Year','Player','Role'])
players_rus[cols]=players_rus[cols].apply(lambda x: x.str.replace(',',''))
players_rus[cols] = players_rus[cols].apply(pd.to_numeric, errors='coerce')

VTB-United-League


C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_16564\1379242875.py:37: FutureWarning: The f

In [36]:
players_esp.to_csv('players_esp.csv',index=False)

In [52]:
players_it.to_csv('players_it.csv',index=False)
players_tur.to_csv('players_tur.csv',index=False)
players_gr.to_csv('players_gr.csv',index=False)
players_fr.to_csv('players_fr.csv',index=False)
players_rus.to_csv('players_rus.csv',index=False)
players_adr.to_csv('players_adr.csv',index=False)

In [12]:
players_esp_adv=get_players_adv(1)
cols = players_esp_adv.columns.drop(['Team','Year','Player'])
players_esp_adv[cols]=players_esp_adv[cols].apply(lambda x: x.str.replace(',',''))
players_esp_adv[cols] = players_esp_adv[cols].apply(pd.to_numeric, errors='coerce')
players_esp_adv.to_csv('players_esp_adv.csv',index=False)

Spanish-ACB
1
2


C:\Users\marli\AppData\Local\Temp\ipykernel_10136\838563138.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)


1
2


C:\Users\marli\AppData\Local\Temp\ipykernel_10136\838563138.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)


1
2


...

1
2


C:\Users\marli\AppData\Local\Temp\ipykernel_10136\838563138.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)


1
2


C:\Users\marli\AppData\Local\Temp\ipykernel_10136\838563138.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)


1
2


In [22]:
players=get_players_adv(2)
cols = players.columns.drop(['Team','Year','Player'])
players[cols]=players[cols].apply(lambda x: x.str.replace(',',''))
players[cols] = players[cols].apply(pd.to_numeric, errors='coerce')
players.to_csv('players_it_adv.csv',index=False)

VTB-United-League


C:\Users\marli\AppData\Local\Temp\ipykernel_10136\2033483725.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_10136\2033483725.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_10136\2033483725.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_10136\2033483725.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(output)
C:\Users\marli\AppData\Local\Temp\ipykernel_10136\2033483725.py:35: FutureWarning: The f

In [ ]:
players=get_players_adv(3)
cols = players.columns.drop(['Team','Year','Player'])
players[cols]=players[cols].apply(lambda x: x.str.replace(',',''))
players[cols] = players[cols].apply(pd.to_numeric, errors='coerce')
players.to_csv('players_tur_adv.csv',index=False)

In [ ]:
players=get_players_adv(4)
cols = players.columns.drop(['Team','Year','Player'])
players[cols]=players[cols].apply(lambda x: x.str.replace(',',''))
players[cols] = players[cols].apply(pd.to_numeric, errors='coerce')
players.to_csv('players_gr_adv.csv',index=False)

In [ ]:
players=get_players_adv(5)
cols = players.columns.drop(['Team','Year','Player'])
players[cols]=players[cols].apply(lambda x: x.str.replace(',',''))
players[cols] = players[cols].apply(pd.to_numeric, errors='coerce')
players.to_csv('players_fr_adv.csv',index=False)

In [ ]:
players=get_players_adv(6)
cols = players.columns.drop(['Team','Year','Player'])
players[cols]=players[cols].apply(lambda x: x.str.replace(',',''))
players[cols] = players[cols].apply(pd.to_numeric, errors='coerce')
players.to_csv('players_adr_adv.csv',index=False)

In [ ]:
players=get_players_adv(7)
cols = players.columns.drop(['Team','Year','Player'])
players[cols]=players[cols].apply(lambda x: x.str.replace(',',''))
players[cols] = players[cols].apply(pd.to_numeric, errors='coerce')
players.to_csv('players_rus_adv.csv',index=False)